In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
google_colab = input('Running on Colab?\n')
if google_colab=='yes' : 
    BASE_DIR = 'University/UC3M/TFM/cnp_repo/neural-processes'

    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')

      # Change to assignment directory ('Lab_Exercises_DASS/Lab_Exercise4' by default)
    import os
    os.chdir(f'/content/drive/My Drive/{BASE_DIR}')
    
else:
    import sys, os 
    BASE_DIR = '/'.join(os.getcwd().split('/')[:-1])
    sys.path.append(BASE_DIR)
    print(f'Top level directory is set to {BASE_DIR}')
    os.chdir(BASE_DIR)

    
# custom imports
from cnp.trainer import RegressionTrainer
from cnp.datageneration import DataGenerator
from cnp.helpers import Helper

import torch
import numpy as np

# import utils
import json
from datetime import date

Running on Colab?
noi
Top level directory is set to /Users/frederik/Google Drive/University/UC3M/TFM/cnp_repo/neural-processes


In [5]:

# we first get the variables for the configuration file
os.environ["CHECKPOINT_DIR"] = "checkpoints"
os.environ["MODEL_PARAMS"] = "default"

CHECKPOINT_DIR = os.environ['CHECKPOINT_DIR']
MODEL_PARAMS = os.environ['MODEL_PARAMS']

if google_colab=='yes':
    config_dir = ''
else:
    config_dir = os.path.join(BASE_DIR,'configs')

with open(os.path.join(config_dir,'1d_singlefunc_config.json')) as f:
    file = f.read()
    default_conf = json.loads(file)



In [6]:

# checking whether CUDA is available 
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
    default_conf['single_function']['experiment_params']['train_on_gpu'] = True
else: 
    print('No GPU available, training on CPU')
    default_conf['single_function']['experiment_params']['train_on_gpu'] = False

No GPU available, training on CPU


In [7]:
default_conf['single_function']['training_params']['kwargs']['data_path'] = os.path.join(
    BASE_DIR,
    default_conf['single_function']['training_params']['kwargs']['data_dir'],
     default_conf['single_function']['training_params']['kwargs']['extension'],
    default_conf['single_function']['training_params']['kwargs']['data_file'])

## We will first try to replicate what we have after one encoding step

In [34]:
import pandas as pd 
import numpy as np
import sys
import matplotlib.pyplot as plt
from cnp.networks import Encoder, create_linear_layer
from cnp.cnp import get_sample_indexes, select_data
from sklearn.linear_model import LinearRegression
X_train, y_train, X_vali, y_vali = Helper.read_and_transform(
    default_conf['single_function']['training_params']['kwargs']['data_path'],
    'target')


In [9]:
batch_size = 50

In [10]:
trainloader = Helper.create_loader(X_train, y_train, batch_size)
valiloader = Helper.create_loader(X_vali, y_vali, batch_size)

In [11]:
xvalues = X_train[:50,:]
funcvalues = y_train[:50]
xvalues = xvalues.unsqueeze(0)
funcvalues = funcvalues[None, :, None]

In [23]:
sample_specs_kwargs = {
            "min_trgts": default_conf['single_function']['experiment_params']['min_funcs'],
            "max_trgts": default_conf['single_function']['experiment_params']['max_funcs'],
            "max_contx": default_conf['single_function']['experiment_params']['max_contx'],
            "min_contx": default_conf['single_function']['experiment_params']['min_contx']}



In [24]:
func_idx, contxt_idx = get_sample_indexes(
    **sample_specs_kwargs,
    dim_observation=default_conf['single_function']['experiment_params']['dim_observation']
)

In [25]:
batch_size_small = xvalues.shape[0]
um_contxt, num_trgt, target_x, target_y, context_x_stacked,context_y_stacked, target_x_stacked = select_data(
            contxt_idx, func_idx, xvalues,
            funcvalues, batch_size_small)

In [26]:
encoder = Encoder(dimx=1, dimy=1, dimr=128, num_layers=3, num_neurons=128)

- When we are taking the mean we say that each context point is equally important 
- When we parametize this function we say that not each point is equally important

In [197]:

class Aggregator(nn.Module):
        def __init__(
            self,
            maxcontxt: int,
            dimout: int,
            num_layers: int,
            num_neurons: int,
            dropout: float = 0) -> None:

            super().__init__()
            self._dimr = maxcontxt
            self._dimout= dimout
            self._hidden_layers = [num_neurons for _ in range(num_layers)]

            _first_layer = [
                nn.Linear(self._dimr, self._hidden_layers[0]),
                nn.ReLU()]
            _hidden_layers = [
                create_linear_layer(self._hidden_layers, i, dropout)
                for i in range(len(self._hidden_layers) - 2)]
            _hidden_layers_flat = [
                element for inner in _hidden_layers for element in inner]

            _last_layer = [
                nn.Linear(self._hidden_layers[-2], self._dimout)]

            self._layers = _first_layer + _hidden_layers_flat + _last_layer

            self._process_input = nn.Sequential(*self._layers)
            
        def forward(self, embedding):
            """
            Parameters
            ----------
            embedding: torch.Tensor: Shape (batch_size*num_contxt, dimr)

            """
            return self._process_input(embedding)


In [165]:
encoding = encoder(context_x_stacked, context_y_stacked)

In [167]:
encoding = torch.transpose(encoding,0,1)

In [184]:
encoding_exp = encoding[None,:,:]

torch.Size([128, 7])

In [200]:
encoding_padded = zero_padding(encoding_exp, 10)

In [194]:
def zero_padding(source, target_dims):
    batch__size, n_observations, n_features = source.size()
    zero_target = torch.zeros(batch__size, n_observations, target_dims)
    zero_target[:,:,:n_features] = source
    return  zero_target
    

In [199]:
aggr = Aggregator(maxcontxt=10, dimout=1, num_layers=4, num_neurons=20)

In [202]:
aggr(encoding_padded).shape

torch.Size([1, 128, 1])

In [208]:
encoding[None, :,:].mean(1).shape

torch.Size([1, 7])